In [52]:
import os
import sys
import subprocess
import json
import rdflib
from rdflib import RDF, RDFS

In [53]:
modelicaPathStr = os.environ['MODELICAPATH']
modelicaJsonPath = os.environ['MODELICAJSONPATH']
modelicaPaths = modelicaPathStr.split(':')
index = 0
for modelicaPath in modelicaPaths:
    subFolders = os.listdir(modelicaPath)
    if 'Buildings' not in subFolders:
        index+=1
        continue
    else:
        break
buildingsLibraryPath = modelicaPaths[index]

In [54]:
cdlPackagePath = os.path.join(buildingsLibraryPath, 'Buildings', 'Controls', 'OBC', 'CDL')
if os.path.exists(cdlPackagePath):
    print(f'CDL package exists at: {cdlPackagePath}')

CDL package exists at: /Users/akprakash/Programming/modelica/modelica-buildings/Buildings/Controls/OBC/CDL


In [55]:
def runModelicaJson(model, outputDir, modelicaJsonPath):
    """This function uses the modelica-json tool to generate instances of a modelica model using the semantic output.

    Parameters:
    -----------
    model : str
            Absolute path to modelica model whose instances will be generated.
    outputDir : str
            Directory where to store the json output.
    """
    appJsPath = os.path.join(modelicaJsonPath, 'app.js')
    result = subprocess.run(['node', '{}'.format(appJsPath), '-f', '{}'.format(model),
                             '-d', '{}'.format(outputDir), '-o', 'semantic'],
                            stdout=subprocess.PIPE, shell=False)

    if result.returncode != 0:
        raise Exception("Error while converting modelica file to json = {0}.".format(result.stdout.decode("utf-8")))

In [56]:
def traverseFolder(folderPath):
    folderContents = os.listdir(folderPath)
    cdlElementaryBlocks = []
    for content in folderContents:
        contentPath = os.path.join(folderPath, content)
        if os.path.isdir(contentPath) and content != 'Validation':
            newCdlElementaryBlocks = traverseFolder(os.path.join(folderPath, content))
            cdlElementaryBlocks = cdlElementaryBlocks + newCdlElementaryBlocks
        else:
            if content != 'package.order' and content != 'package.mo':
                cdlElementaryBlocks.append(os.path.join(folderPath, content))
    return cdlElementaryBlocks

In [70]:
def parseCdlInstances(instances, blockName, graph, ns231p):
    instanceCategoryDict = {}
    for instance in instances:
        instanceDict = instances.get(instance)
        if instanceDict == None:
            # TODO: handle later
            pass  
        elif instanceDict.get('type', '') == 'enumeration': 
            # TODO: handle this
            pass
        elif instanceDict.get('type', '') == 'long_class_specifier':
            #do nothing, because its not an instance of the block
            pass
        elif instanceDict.get('type', '') == 'element':
            if instanceDict.get('type_prefix', '') == 'parameter': ## todo: handle constant
                value = {
                    instance: {}
                }
                print(blockName)
                graph.add((ns231p[blockName], ns231p['hasParameter'], ns231p[f'{blockName}#{instance}']))
#                 if 'parameters' in instanceCategoryDict:
#                     instanceCategoryDict['parameters'].append(value)
#                 else:
#                     instanceCategoryDict['parameters'] = [value]
            elif instanceDict.get('type_prefix', '') == '':
                if instanceDict.get('type_specifier', '').endswith('IntegerInput') or instanceDict.get('type_specifier', '').endswith('BooleanInput') or instanceDict.get('type_specifier', '').endswith('RealInput'):
                    value = {
                        instance: {}
                    }
                    graph.add((ns231p[blockName], ns231p['hasInput'], ns231p[f'{blockName}#{instance}']))
                    graph.add((ns231p[f'{blockName}#{instance}'], rdflib.RDF['type'], ns231p['InputConnector']))
#                     if 'inputs' in instanceCategoryDict:
#                         instanceCategoryDict['inputs'].append(value)
#                     else:
#                         instanceCategoryDict['inputs'] = [value]
                if instanceDict.get('type_specifier', '').endswith('IntegerOutput') or instanceDict.get('type_specifier', '').endswith('BooleanOutput') or instanceDict.get('type_specifier', '').endswith('RealOutput'):
                    value = {
                        instance: {}
                    }
                    graph.add((ns231p[blockName], ns231p['hasOutput'], ns231p[f'{blockName}#{instance}']))

                    graph.add((ns231p[f'{blockName}#{instance}'], rdflib.RDF['type'], ns231p['OutputConnector']))
#                     if 'outputs' in instanceCategoryDict:
#                         instanceCategoryDict['outputs'].append(value)
#                     else:
#                         instanceCategoryDict['outputs'] = [value]
    return instanceCategoryDict, graph

In [71]:
cdlCore = rdflib.Graph()
ns231p = rdflib.Namespace('https://data.ashrae.org/231P/')
cdlCore.bind('s231', ns231p)

In [72]:
blocks = traverseFolder(cdlPackagePath)
outputDir = 'temp'
for block in blocks:
    runModelicaJson(block, outputDir, modelicaJsonPath)
    blockName = block.split(os.path.join(buildingsLibraryPath, 'Buildings', 'Controls', 'OBC')+os.sep)[1]
    blockName = blockName.split(".mo")[0]
    objectsOutput = os.path.join(outputDir, 'objects', 'Buildings', 'Controls', 'OBC', blockName+'.json')
    if os.path.exists(objectsOutput):
        with open(objectsOutput) as fp:
            objects = json.load(fp)
    else:
        objects = {}
    blockName = blockName.replace(os.sep, '.')
    blockName = f'CXF.{blockName.split("CDL.")[-1]}'
    print(blockName, instanceDict)
    cdlCore.add((ns231p[blockName], RDFS.subClassOf, ns231p['ElementaryBlock']))
    instanceDict, cdlCore = parseCdlInstances(objects.get('instances', {}), blockName, cdlCore, ns231p)
    
    
    
    

CXF.Types.SimpleController {}
CXF.Types.Smoothness {}
CXF.Types.Extrapolation {}
CXF.Types.ZeroTime {}
CXF.Conversions.BooleanToInteger {}
CXF.Conversions.BooleanToInteger
CXF.Conversions.BooleanToInteger
CXF.Conversions.BooleanToReal {}
CXF.Conversions.BooleanToReal
CXF.Conversions.BooleanToReal
CXF.Conversions.IntegerToReal {}
CXF.Conversions.RealToInteger {}
CXF.Conversions.Validation {}
CXF.Integers.Subtract {}
CXF.Integers.Add {}
CXF.Integers.LessEqualThreshold {}
CXF.Integers.LessEqualThreshold
CXF.Integers.Equal {}
CXF.Integers.Max {}
CXF.Integers.OnCounter {}
CXF.Integers.OnCounter
CXF.Integers.Switch {}
CXF.Integers.Abs {}
CXF.Integers.GreaterEqual {}
CXF.Integers.Multiply {}
CXF.Integers.LessEqual {}
CXF.Integers.GreaterEqualThreshold {}
CXF.Integers.GreaterEqualThreshold
CXF.Integers.LessThreshold {}
CXF.Integers.LessThreshold
CXF.Integers.Min {}
CXF.Integers.GreaterThreshold {}
CXF.Integers.GreaterThreshold
CXF.Integers.Sources.Pulse {}
CXF.Integers.Sources.Pulse
CXF.Intege

In [74]:
cdlCore.serialize('CXF-core.ttl')

<Graph identifier=Na3f34ba2e115469fb870bd08b3de7d6b (<class 'rdflib.graph.Graph'>)>

## questions for MW & JH:
- Validation folder? 
- cdl.reals.sources.validation
- cdl.discrete.examples?